In [46]:
#libraries
import numpy as np
import matplotlib as plt

In [ ]:
# tef,dtt,a: array [0..1501] of longint# 
# tav,ms,Ω,djs,djs1,μ,φ,Rsat,Xsp,Xsp1,PerSat,dφs,t2,t1,Jsc,Jr1,Jr2,j12,tpl1,tpl2:array [0..1500] of real# 
# t1s,t2s,te1s,te2s,Jadd1,Jadd2,k: array [1..10,0..1501] of real#
# Pp,Ps,Rp,Ap,m,Mp,φp,dφp,dt1:longint# 
# dt,n,ts,s,t:longint# 
# Rs,xp,xp1,Xplo,Xpl,Xpln,Xpln1,Xpl1o,G,Mstar,Mplanet,Ωp,F,f1,Alb:real# 
# delt,tint,i:integer# 
# delj,dttav,jav,sigmajdtt,sigmaj,sigmadtt,Jres,Jcut,Jcuto:real#
# PpObs,t0,Korr,TTVmax,TTVmin:real# 
# PerP:real#

# Jo:array [0..1500,-70000..70000] of real#
# V1,V2,V3,V4,V5,L,S3,djp1,djp,djp0:real#
# Rz,Aps,Ass:longint# 

# xs,xs1,dJ:real# 

# te1,te2:integer# 

# nn,sumn,ntav,sumtav,Delta,DeltaPp,Deltat0:real# 

In [81]:
m=1;
ms=[None]*3
a=[None]*3
Rsat=[None]*3

G=6.67*10**(-11)#
Mstar=1.99*10**30#
Mplanet=5.99*10**24#

Mp=1#{Масса планеты в относительных единицах}
ms[0]=1/81.3#{Масса спутника-1 в массах планеты}
ms[1]=1/116.14#{Масса спутника-2 в массах планеты}
ms[2]=1/98#
Ap=149598000# {Большая полуось орбиты планеты в км} 
Pp=31556952# {Период обращения планеты вокруг звезды в сек}
Ps=245000# {Период обращения спутника вокруг планеты в сек} 
Rp=6400# {Радиус планеты в км} 
Rs=695000# {Радиус звезды в км} 

a[0]=384000#{Радиус орбиты спутника-1 в км}
a[1]=192000#{Радиус орбиты спутника-2 в км}
a[2]=699000#

Rsat[0]=1738# {Радиус спутника-1 в км} 
Rsat[1]=1225# {Радиус спутника-2 в км} 
Rsat[2]=1950#
Alb=0.3#
s=3# {Число транзитов системы спутники-планета}  
tint=2000# {Время усреднения яркости до/после затмения в сек} 
delJ=0.000007#{Погрешность измения яркости, случайная добавка принимает значения от delJ до +delJ, всегда равна нулю при delt=0} 
delt=6# {Погрешность изменения времени транзита, принимает значения от -delt до +delt, в сек} 
φp=0# what is it?
Ωp=np.sqrt(G*Mstar/1000000000/Ap/Ap/Ap)#
djp=(Rp/Rs)**2#
djp0=Rp/Rs#
PerP=2*np.pi/Ωp#
Ω=[None]*m
djs=[None]*m
μ=[None]*m
dφs=[None]*m
PerSat=[None]*m
φ=[None]*m
dφs=[None]*m
t1=[None]*s   
t2=[None]*s
Xsp=[None]*s
Xsp1=[None]*s
djs1=[None]*m
tpl1=[None]*s
tpl2=[None]*s
w1 = s
T=70000
t1s = [[0 for x in range(w1)] for y in range(m)] 
t2s = [[0 for x in range(w1)] for y in range(m)] 
te1s = [[0 for x in range(w1)] for y in range(-T,T)] 
te2s= [[0 for x in range(w1)] for y in range(-T,T)] 
Jo= [[0 for x in range(w1)] for y in range(-T-1,T)] 

In [84]:
for i in range(m):
    Ω[i]=np.sqrt(G*Mplanet/1000000000/a[i]/a[i]/a[i])# {Доп. коэфф. необходим при переводе из км в метры}
    djs[i]=Rsat[i]**2/Rs**2# {Изменение яркости звезды во время транзита i-го спутника}
    μ[i]=(Mp/ms[i])# {Отношение масс i-го спутника к массе планеты.}
    print(Ω[i])
    print('djs= ', djs[i])#
    print('μ=', μ[i])#
    PerSat[i]=2*np.pi/Ω[i]#
    dφs[i]=(Ω[i]*PerP/2/np.pi-int(Ω[i]*PerP/2/np.pi))# Phase shift?
    print('Ps-',i,'=',PerSat[i])#
    print('dφs-',i,'=',dφs[i])#
    
print('Pp=',PerP)#

2.6563144917024576e-06
djs=  6.253597639873713e-06
μ= 81.3
Ps- 0 = 2365377.039054074
dφs- 0 = 0.34070305677683876
Pp= 31555792.695338428


In [83]:
#we have to understand this loop

# print(len(Jo))

for n in range(s):
    #{n-transit's number}

    for i in range(m):
        φ[i]=2*np.pi*(n*dφs[i]-int(n*dφs[i]))# current phase of each satellite?
        t=-T#
        
    while t<T:#by second iteration
        t=t+5#
#         dt=dt+5
        Xplo=0#
        for i in range(m):#Contribution of every satellite to planet's position
            Xplo=Xplo-(a[i]/(μ[i]+1)*np.sin(Ω[i]*t+φ[i]))
            Xp=0#
            Xp=Xplo+Ap*np.sin(Ωp*t+φp)#{координата центра планеты в момент t}
            Xp1=0#
            Xp1=Xplo+Ap*np.sin(Ωp*(t+1)+φp)#{координата центра планеты в момент t+1}
            
        #every time check the condition:
        
        if (-Xp>Rs+Rp) and (-Xp1<Rs+Rp):
            t1[n]=t# {Условие вступления центра планеты на диск звезды, t1[n]-время начала n-го транзита в сек}
        if (Xp<Rs+Rp) and (Xp1>Rs+Rp):
            t2[n]=t# {Условие схода центра планеты с диска звезды, t2[n]-время окончания n-го транзита в сек}

#{Транзиты планеты без участия спутников}

        Xpln=Ap*np.sin(Ωp*t+φp)
        Xpln1=Ap*np.sin(Ωp*(t+1)+φp)#
        if (-Xpln>Rs+Rp) and (-Xpln1<Rs+Rp):
            tpl1[n]=t# {Условие вступления центра планеты на диск звезды, tpl1[n]-время начала n-го транзита в сек}
        if (Xpln<Rs+Rp) and (Xpln1>Rs+Rp):
            tpl2[n]=t# {Условие схода центра планеты с диска звезды, tpl2[n]-время окончания n-го транзита в сек} 
  
    
        for i in range(m):
            Xsp[i]=Xp+a[i]*(μ[i]/(μ[i]+1))*np.sin(Ω[i]*t+φ[i])# {координата центра спутника в момент t}   
            Xsp1[i]=Xp1+a[i]*(μ[i]/(μ[i]+1))*np.sin(Ω[i]*(t+1)+φ[i])# {координата центра спутника в момент t+1}
            
            if (-Xsp[i]>Rs) and (-Xsp1[i]<Rs):
                
                t1s[i][n]=t # {Условие вступления центра спутника на диск звезды, t1s[n]-время начала n-го транзита в сек} 
            if (Xsp[i]<Rs) and (Xsp1[i]>Rs):
                t2s[i][n]=t# {Условие схода центра спутника с диска звезды, t2[n]-время окончания n-го транзита в сек}
        Jo[t][n]=1
        
        V3=0 # what is it?
        for i in range(m):
            if (-Xsp[i]<(Rs+Rsat[i])) and (-Xsp[i]>Rs):
                V1=1
            else:
                V1=0
            if  (-Xsp[i]>(Rs-Rsat[i])) and (-Xsp[i]<Rs):
                V2=1 
            else:
                V2=0#
            if  (Xsp[i]>(Rs-Rsat[i])) and (Xsp[i]<Rs):
                V4=1
            else:
                V4=0#
            if   (Xsp[i]>Rs) and (Xsp[i]<(Rs+Rsat[i])):
                V5=1
            else:
                V5=0#
            l=0
            if (V1>0) or (V2>0) or (V4>0) or (V5>0):
                l=np.sqrt((Rsat[i]*Rsat[i])-(abs(Xsp[i])-Rs)*(abs(Xsp[i])-Rs))
                
            S3=np.arccos(abs(-Rs+abs(Xsp[i]))/Rsat[i])*Rsat[i]*Rsat[i]-l*abs(-Rs+abs(Xsp[i]))#
            djs1[i]=(S3/(np.pi*Rs*Rs))# 
            Jo[t][n]=Jo[t][n]-(djs1[i]*(V1+V5)+(djs[i]-djs1[i])*(V2+V4)+djs[i]*V3)#
        
#  {Jo[n,t]=1-(1-0.65+0.65*np.sqrt(abs(1-(abs(Xsp[i]))*(abs(Xsp[i]))/Rs/Rs)))*(djs1[i]*(V1+V5)+(djs[i]-djs1[i])*(V2+V4)+djs[i]*V3)#}
        V1=0#
        V2=0#
        V3=0#
        V4=0#
        V5=0#
        if (-Xp<(Rs+Rp)) and (-Xp>Rs):
            V1=1 
        else:
            V1=0#
        if  (-Xp>(Rs-Rp)) and (-Xp<Rs):
            V2=1
        else:
            V2=0#
        if (-Xp<(Rs-Rp)) and (Xp<(Rs-Rp)):
            V3=1
        else:
            V3=0#
        if  (Xp>(Rs-Rp)) and (Xp<Rs):
            V4=1   
        else:
            V4=0
        if   (Xp>Rs) and (Xp<(Rs+Rp)):
            V5=1
        else: 
            V5=0#
        
        l=0
        if V1>0 or V2>0 or V4>0 or V5>0:
            l=np.sqrt((Rp*Rp)-(abs(Xp)-Rs)*(abs(Xp)-Rs))
            S3=np.arccos(abs(-Rs+abs(Xp))/Rp)*Rp*Rp-l*abs(-Rs+abs(Xp))#
            
        djp1=(S3/(np.pi*Rs*Rs))#
        Jo[t][n]=Jo[t][n]-(djp1*(V1+V5)+(djp-djp1)*(V2+V4)+djp*V3)#
        #  {Jo[n,t]=Jo[n,t]-(1-0.65+0.65*np.sqrt(abs(1-(abs(Xp))*(abs(Xp))/Rs/Rs)))*(djp1*(V1+V5)+(djp-djp1)*(V2+V4)+djp*V3)#}

#             {print(' (',t,'#',Jo[n,t],')')#
#         print('n=',n,' t1= ',t1[n])#}


     # end while-a
    print(t1[n])
    print(t2[n])
    print(tpl1[n])
    print(tpl2[n])
#     for i in range(m):
#         te1s[n][i]=t1[n]-t1s[n][i]#{Продолж. нахождения спутников на диске звезды до и после транзита планеты}
#         te2s[n][i]=t2s[n][i]-t2[n]#

C:\Users\rtut6\Anaconda3\lib\site-packages\ipykernel_launcher.py:74: RuntimeWarning: invalid value encountered in arccos


None
None
None
None
None
None
None
None
None
None
None
None


In [49]:
       
        
        
        
        
        
# # end n cycle



# for n in range(s):
#     for t in range(np.round(t1[n]-tint),np.round(t1[n]):
#                    Jr1[n]=Jr1[n]+Jo[n,t]#

#     for t=np.round(t2[n]) to np.round(t2[n]+tint) do begin
#     Jr2[n]=Jr2[n]+Jo[n,t]#

# for n in range(s):
#     print(n,'Jr1= ',Jr1[n])#
#     print(n,'Jr2= ',Jr2[n])# 
#     Jr1[n]=Jr1[n]*5/tint#
#     Jr2[n]=Jr2[n]*5/tint#
#     J12[n]=Jr2[n]/Jr1[n]# {отношение j2[n] и j1[n]}
#     {j12[n]=j12[n]+2*delj*(Random(delt+1)-(delt+1)/2)/(delt+1)#}
#     print('J12[',n,']=',J12[n],'#')#


 

# # for n=0 to s do t1[n]=t1[n]+2*Random(delt+1)-(delt+1)#{Эта строчка и две следующие - случайные погрешности
# # for n=0 to s do t2[n]=t2[n]+2*Random(delt+1)-(delt+1)# }

    
# for n=0 to s do tav[n]=np.round((t1[n]+t2[n])/2)# {tav[n]-время середины n-того

# nn=0# for n=0 to s do nn=nn+n*n# {отсюда и до конца абзаца составление эфемериды планеты методом наименьших квадратов
# sumn=0# for n=0 to s do sumn=sumn+n# 
# ntav=0# for n=0 to s do ntav=ntav+n*tav[n]# 
# sumtav=0# for n=0 to s do sumtav=sumtav+tav[n]# 
# Delta=nn*(s+1)-sumn*sumn# 
# DeltaPp=ntav*(s+1)-sumtav*sumn# 
# Deltat0=nn*sumtav-sumn*ntav# 
# PpObs=DeltaPp/Delta# 
# t0=Deltat0/Delta# }

# print('Pp=', PpObs:9:9,' t0=',t0:6:6)# {ppObs-наблюдаемый период обращения планеты вокруг звезды, на основе известных времен транзитов в сек} 
# {t0- постоянная, входящая в эферемениду, измеряется в сек} 

# {for n=0 to s do tpl1[n]=PpObs*(n-arcnp.sin(Rs/Ap)/2/pi)#
# for n=0 to s do tpl2[n]=PpObs*(n+arcnp.sin(Rs/Ap)/2/pi)#}
# {for n=0 to s do dtt[n]=np.round(tav[n])-tef[n]# {dtt[n] - вариация времени середины n-того транзита планеты в сек
# for n=0 to s do tef[n]=np.round(n*PpObs+t0)# tef[n] - время середины n-того транзита планеты согласно эфемериде в сек} 

# for n=0 to s do dtt[n]=np.round((t1[n]-tpl1[n]+t2[n]-tpl2[n])/2)# {dtt[n] - вариация времени середины n-того транзита планеты в сек} 
# print('dtt[n]=','  ',dtt[n])#

# {for n=0 to s do print('TDV[',n,']=',-(tpl2[n]-tpl1[n])+(t2[n]-t1[n]),'#')#}

# for n=0 to s do print('[',(t1[n]-tpl1[n]+t2[n]-tpl2[n])/2,',',(1-1/J12[n])*1000000:3:3,']',',')# 

# {for n=0 to s do print('[',(t1[n]-tpl1[n]+t2[n]-tpl2[n])/2,',', -(tpl2[n]-tpl1[n])+(t2[n]-t1[n]),']',',')#}

# {for n=0 to s do print('TTV[',n,']=',trunc((t1[n]-tpl1[n]+t2[n]-tpl2[n])/2),'#')#}
# jav=0# for n=0 to s do jav=jav+j12[n]/(s+1)# print('jav=',jav:8:8)#
# dttav=0# for n=0 to s do dttav=dttav+dtt[n]/(s+1)# print('dttav=',dttav:8:4)#
# Sigmaj=0# for n=0 to s do Sigmaj=Sigmaj+j12[n]*j12[n]-jav*jav# Sigmaj=np.sqrt(Sigmaj)#
# Sigmadtt=0# for n=0 to s do Sigmadtt=Sigmadtt+dtt[n]*dtt[n]-dttav*dttav# Sigmadtt=np.sqrt(Sigmadtt)#
# Sigmajdtt=0# for n=0 to s do Sigmajdtt=Sigmajdtt+(j12[n]-jav)*(dtt[n]-dttav)# 
# print(Sigmaj,' ',Sigmadtt,' ',Sigmajdtt)# 
# Korr=Sigmajdtt/Sigmaj/Sigmadtt# 
# print('Коэффициент корреляции, Korr=', Korr:4:3)# 
# end.



In [2]:
Ω=1
print(Ω)

1
